In [11]:
import requests
import time,re
import json
from kafka import KafkaProducer
from kafka import TopicPartition
from bs4 import BeautifulSoup
from selenium import webdriver
import sched

In [12]:
def get_web_page(url):
    chromedriver = "/Users/LeoWang/Downloads/chromedriver"

    driver   = webdriver.Chrome(chromedriver)
    driver.get(url)

    for i in range(1,10):
        driver.execute_script('window.scroll(0,100000)')
        time.sleep(2)

    pagesource = driver.page_source
    driver.close()
    return pagesource

In [13]:
def get_articles(dom, date, kafka_topic):
    
    kafka_host='192.168.99.100' # host
    kafka_port=9092 # port
    
    producer = KafkaProducer(bootstrap_servers=['{kafka_host}:{kafka_port}'.format(
        kafka_host=kafka_host,
        kafka_port=kafka_port
    )])
    
    yahoo_soup = BeautifulSoup(dom,'html.parser')
    links = []
    finance = yahoo_soup.find_all('li', 'js-stream-content')

    for info in finance:
        link = ""
        try:
            test = info.find_all('div', 'controller')
            if test:
                continue

            title = info.find('a').text
            #print(title)

            #print('---')

            eassy = info.find('p').text
            #print(eassy)

            #print("===")
            """
            link = info.find_all('a', href=True)[0]

            if link.get('href') != '#':
                links.append(base_url + link.get("href"))
                print(title)
                print(base_url + link.get("href"))
                print('===')
            """
            producer.send(kafka_topic, title.encode('utf-8'))
        except:
            link = None
    producer.send(kafka_topic, b'done')
    producer.flush()

In [14]:
def main():
    schedule = sched.scheduler ( time.time, time.sleep )
    current_page = get_web_page('https://tw.news.yahoo.com/politics/')
    current_page2 = get_web_page('https://tw.news.yahoo.com/entertainment/')
    kafka_topic='poli'
    kafka_topic2='ente'
    if current_page:
        today = time.strftime("%m/%d").lstrip('0') 
        get_articles(current_page, today, kafka_topic)
    if current_page2:
        today = time.strftime("%m/%d").lstrip('0')
        get_articles(current_page2, today, kafka_topic2)
        
    schedule.enter(10800,0,main,()) 
    print('wait 3 hours...')
    schedule.run()

In [ ]:
main()

wait 3 hours...
